# Streets to Zips
The purpose of this notebook is to get the zip codes for the corresponding street address using the MapQuest Search API.

**Ednalyn C. De Dios**\
_Data Scientist_

# Credits / References

Codeup Ada One (Datathon team):

- Dd (Ednalyn C. De Dios)
- Joseph Burton
- Sandy Graham

MapQuest Search API:\
https://developer.mapquest.com/documentation/search-api/v2/postal-codes/

Data Source:\
https://sites.google.com/respec.com/smartsa-datathon-2019/

Github Repository (Datathon 2019):\
https://github.com/codeupada1/san-antonio-datathon-2019

In [37]:
# for manipulating dataframes
import pandas as pd
from pandas import DataFrame

# For GET request from the Geocoding API
from requests import get
import json

# There's a limit of 15,000 GET requests per day.
# Sign up at https://developer.mapquest.com/documentation
# to get another account and a new set of keys.
# API_KEY = 'Zd5jr3WZm1PbGobgPDHzLz9LEFDaco1V' # This one has reached max limit.
API_KEY = 'UjvBNaD5Wzqj4G2czUAWNakf7GjhvMEv'

# to print out all the outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# suppress all warnings
import warnings
warnings.filterwarnings('ignore')

In [8]:
# Read the excel file
df = pd.read_excel('../data/in/SAWS_SSOs_2009-2018Mar_UploadData.xlsx')

df.shape
df.head(3)

(3183, 57)

,SSO_ID,INSPKEY,SERVNO,REPORTDATE,SPILL_ADDRESS,SPILL_ST_NAME,TOTAL_GAL,GALSRET,GAL,SPILL_START,SPILL_STOP,HRS,CAUSE,COMMENTS,ACTIONS,WATERSHED,UNITID,UNITID2,DISCHARGE_TO,DISCHARGE_ROUTE,COUNCIL_DISTRICT,FERGUSON,Month,Year,Week,EARZ_ZONE,Expr1029,PIPEDIAM,PIPELEN,PIPETYPE,INSTYEAR,DWNDPTH,UPSDPTH,Inches_No,RainFall_Less3,SPILL ADDRESS,SewerAssetExp,NUM_SPILLS_COMPKEY,NUM_SPILLS_24MOS,PREVSPILL_24MOS,UNITTYPE,ASSETTYPE,LASTCLND,ResponseTime,ResponseDTTM,Public Notice,TIMEINT,Root_Cause,STEPS_TO_PREVENT,SPILL_START_2,SPILL_STOP_2,HRS_2,GAL_2,SPILL_START_3,SPILL_STOP_3,HRS_3,GAL_3
0,6582,567722.0,NaN,2019-03-10,3200,THOUSAND OAKS DR,2100,2100.0,2100.0,3/10/2019 1:16:00 PM,3/10/2019 2:40:00 PM,1.400000,Grease,"Spill ContainedReturned to SystemArea Cleaned and DisinfectedFlushed Area with H2O, Unstopped Main,",CLEANED MAIN,SALADO CREEK,66918,66917,STREET,None,NaN,172A2,3,2019,11,0.0,NaN,8.0,16.55,PVC,1997.0,NaN,NaN,NaN,NaN,3200 THOUSAND OAKS DR,NaT,1,1.0,NaT,GRAVITY,Sewer Main,NaT,0.450000,2019-03-10 13:43:00,False,24.0,NaN,NaN,NaN,NaN,0.00,0.0,NaN,NaN,0.0,0.0
1,6583,567723.0,NaN,2019-03-10,6804,S FLORES ST,80,0.0,80.0,3/10/2019 2:25:00 PM,3/10/2019 3:45:00 PM,1.333333,Grease,Spill ContainedArea Cleaned and Disinfected,CLEANED MAIN,DOS RIOS,24250,24193,STORMDRAIN,None,3.0,251A3,3,2019,11,0.0,NaN,8.0,157.00,PVC,1988.0,NaN,NaN,NaN,NaN,6804 S FLORES,NaT,1,1.0,NaT,GRAVITY,Sewer Main,NaT,1.083333,2019-03-10 15:30:00,False,120.0,NaN,NaN,NaN,NaN,0.00,0.0,NaN,NaN,0.0,0.0
2,6581,567714.0,NaN,2019-03-09,215,AUDREY ALENE DR,79,0.0,10.0,3/9/2019 6:00:00 PM,3/9/2019 7:30:00 PM,1.500000,Structural,"Spill ContainedArea Cleaned and DisinfectedFlushed Area with H2O, Unstopped Main,",CLEANED MAIN,DOS RIOS,2822,3351,ALLEY,None,1.0,190E4,3,2019,10,0.0,NaN,8.0,350.00,CP,1955.0,NaN,NaN,NaN,NaN,215 Audrey Alene Dr,NaT,1,1.0,NaT,GRAVITY,Sewer Main,NaT,1.000000,2019-03-09 19:00:00,False,24.0,NaN,NaN,03/10/2019 09:36,03/10/2019 10:45,1.15,69.0,NaN,NaN,0.0,0.0


In [9]:
# trim the dataframe to just the two columns that we need to make the street address
df = df[['SPILL_ADDRESS', 'SPILL_ST_NAME']]

# let's create a new column to hold the street address
df['street_address'] = df['SPILL_ADDRESS'].map(str)+ ' ' + df['SPILL_ST_NAME']

# let's get rid of unnecessary columns
df.drop(columns=['SPILL_ADDRESS', 'SPILL_ST_NAME'], inplace=True)

In [16]:
df.head()

,street_address
0,3200 THOUSAND OAKS DR
1,6804 S FLORES ST
2,215 AUDREY ALENE DR
3,3602 SE MILITARY DR
4,100 PANSY LN


In [17]:
# let's transform to title case so it wouldn't scream as much
df['street_address'] = df.street_address.str.title()

In [18]:
df.head()

,street_address
0,3200 Thousand Oaks Dr
1,6804 S Flores St
2,215 Audrey Alene Dr
3,3602 Se Military Dr
4,100 Pansy Ln


In [32]:
addresses = df.street_address

In [19]:
def get_geocode(url):
    """
    This function takes a url and returns a csv that
    contains the lat and long of the provided
    street address.
    """
    response = get(url)
    return response.text

In [22]:
response = (get_geocode('https://www.mapquestapi.com/geocoding/v1/address?key=Zd5jr3WZm1PbGobgPDHzLz9LEFDaco1V&inFormat=kvp&outFormat=json&location=3403+Kildare+Ave&thumbMaps=false&delimiter=%2C'))

obj = json.loads(response)
zip_code = obj['results'][0]['locations'][0]['postalCode']
lat = obj['results'][0]['locations'][0]['latLng']['lat']
long = obj['results'][0]['locations'][0]['latLng']['lng']

In [25]:
obj

{'info': {'statuscode': 0,
  'copyright': {'text': '© 2021 MapQuest, Inc.',
   'imageUrl': 'http://api.mqcdn.com/res/mqlogo.gif',
   'imageAltText': '© 2021 MapQuest, Inc.'},
  'messages': []},
 'options': {'maxResults': -1, 'thumbMaps': False, 'ignoreLatLngInput': False},
 'results': [{'providedLocation': {'location': '3403 Kildare Ave'},
   'locations': [{'street': '3403 Kildare Ave',
     'adminArea6': '',
     'adminArea6Type': 'Neighborhood',
     'adminArea5': 'San Antonio',
     'adminArea5Type': 'City',
     'adminArea4': 'Bexar',
     'adminArea4Type': 'County',
     'adminArea3': 'TX',
     'adminArea3Type': 'State',
     'adminArea1': 'US',
     'adminArea1Type': 'Country',
     'postalCode': '78223-2423',
     'geocodeQualityCode': 'P1AAA',
     'geocodeQuality': 'POINT',
     'dragPoint': False,
     'sideOfStreet': 'L',
     'linkId': 'r43223937|p272882722|n58758857',
     'unknownInput': '',
     'type': 's',
     'latLng': {'lat': 29.374482, 'lng': -98.433155},
     'di

In [26]:
zip_code

'78223-2423'

In [27]:
lat

29.374482

In [28]:
long

-98.433155

In [29]:
def get_zip(url):
    """
    This function takes a url with a street address on it
    and returns the zip code for that address along with
    latitude and longitude coordinates.
    """
    response = (get_geocode(url))
    obj = json.loads(response)
    return  obj['results'][0]['locations'][0]['postalCode'], obj['results'][0]['locations'][0]['latLng']['lat'], obj['results'][0]['locations'][0]['latLng']['lng']

In [31]:
get_zip('https://www.mapquestapi.com/geocoding/v1/address?key=Zd5jr3WZm1PbGobgPDHzLz9LEFDaco1V&inFormat=kvp&outFormat=json&location=3403+Kildare+Ave&thumbMaps=false&delimiter=%2C')

('78223-2423', 29.374482, -98.433155)

In [57]:
get_zip('https://www.mapquestapi.com/geocoding/v1/address?key=Zd5jr3WZm1PbGobgPDHzLz9LEFDaco1V&inFormat=kvp&outFormat=json&location=100+ Military+Plaza&thumbMaps=false&delimiter=%2C')

('78205-2425', 29.424957, -98.494629)

In [35]:
list_addresses = []
for address in addresses:
    cleaned_address = address.replace(' ', '+')
    url = 'https://www.mapquestapi.com/geocoding/v1/address?key=' + API_KEY + '&inFormat=kvp&outFormat=json&location=' + cleaned_address + '+San+Antonio+TX' + '&thumbMaps=false&delimiter=%2C'
    list_addresses.append(get_zip(url))

In [40]:
list_addresses[:10]

[('78247-3317', 29.572974, -98.436752),
 ('78221-1902', 29.352534, -98.489222),
 ('78216-6610', 29.50326, -98.503986),
 ('78223-4582', 29.353559, -98.425234),
 ('78209', 29.487718, -98.435759),
 ('78210', 29.388004, -98.473421),
 ('78245-1147', 29.434926, -98.675149),
 ('78223-1367', 29.382693, -98.466861),
 ('78228-5806', 29.447661, -98.579132),
 ('78201-5445', 29.45436, -98.519284)]

In [45]:
df_locations = pd.DataFrame()

In [46]:
df_locations['addresses']  = addresses.tolist()

In [47]:
df_locations.head(10)

,addresses
0,3200 Thousand Oaks Dr
1,6804 S Flores St
2,215 Audrey Alene Dr
3,3602 Se Military Dr
4,100 Pansy Ln
5,3200 S Hackberry St
6,9910 Sugarloaf Dr
7,3507 Piedmont Ave
8,349 Alicia
9,1502 W Mistletoe Ave


In [48]:
df_temp = DataFrame.from_records(list_addresses)

In [53]:
df_temp.head()

,0,1,2
0,78247-3317,29.572974,-98.436752
1,78221-1902,29.352534,-98.489222
2,78216-6610,29.503260,-98.503986
3,78223-4582,29.353559,-98.425234
4,78209,29.487718,-98.435759


In [50]:
df_locations['zip']  = df_temp[0].tolist()
df_locations['lat']  = df_temp[1].tolist()
df_locations['long']  = df_temp[2].tolist()

In [52]:
df_locations.head()

,addresses,zip,lat,long
0,3200 Thousand Oaks Dr,78247-3317,29.572974,-98.436752
1,6804 S Flores St,78221-1902,29.352534,-98.489222
2,215 Audrey Alene Dr,78216-6610,29.503260,-98.503986
3,3602 Se Military Dr,78223-4582,29.353559,-98.425234
4,100 Pansy Ln,78209,29.487718,-98.435759


In [54]:
df_locations.to_csv('../data/out/geocodes.csv')